# Fit and Food
## Data analysis for the optimal location of a Gym-Restaurant


### Imports of packages

In [1]:
import numpy as np 
import pandas as pd # library for data analsysis

import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

print('Libraries imported.')

Libraries imported.


### Use NYC data

In [17]:
url_nyc_json = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/labs/newyork_data.json'
data_nyc_json = requests.get(url_nyc_json).json()

print('Data downloaded!')

Data downloaded!


In [19]:
data_nyc_json 
neighborhood_data = data_nyc_json['features']
neighborhood_data[0]

# define the dataframe columns
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)

for data in neighborhood_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)

In [20]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(neighborhoods['Borough'].unique()),
        neighborhoods.shape[0]
    )
)

The dataframe has 5 boroughs and 306 neighborhoods.


### Use Geocoder to obtain the latitude and longitude for New York City.

In [3]:
address = 'New York City, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude_nyc = location.latitude
longitude_nyc = location.longitude
print('The geograpical coordinate of New York City are {}, {}.'.format(latitude_nyc, longitude_nyc))

The geograpical coordinate of New York City are 40.7127281, -74.0060152.


### Create the base map for NYC

In [21]:
# create map of New York using latitude and longitude values
map_newyork = folium.Map(location=[latitude_nyc, longitude_nyc], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_newyork)  
    
map_newyork

### Connect with Foursquare API

#### Define FourSquare credentials

In [5]:
CLIENT_ID = 'X35R4UEFHPO1AP1TIOROROAVQNJOQGNWXMUDBYWBX4GKWXR0' # your Foursquare ID
CLIENT_SECRET = 'YYEPGJSNX35I3BOHYLAEOKHLC0YOIRS5SHERM5GYQYU4PT3H' # your Foursquare Secret
VERSION = '20201019' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: X35R4UEFHPO1AP1TIOROROAVQNJOQGNWXMUDBYWBX4GKWXR0
CLIENT_SECRET:YYEPGJSNX35I3BOHYLAEOKHLC0YOIRS5SHERM5GYQYU4PT3H


### Initial Get request for the gyms surrounding the center of NYC

In [6]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius

#Try Get request with NYC center data
neighborhood_latitude = latitude_nyc
neighborhood_longitude = longitude_nyc
category_ids = "4bf58dd8d48988d175941735" # [gym, food]
#first try:
# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}&categoryId={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT,
    category_ids
    )
url # display URL


'https://api.foursquare.com/v2/venues/explore?&client_id=X35R4UEFHPO1AP1TIOROROAVQNJOQGNWXMUDBYWBX4GKWXR0&client_secret=YYEPGJSNX35I3BOHYLAEOKHLC0YOIRS5SHERM5GYQYU4PT3H&v=20201019&ll=40.7127281,-74.0060152&radius=500&limit=100&categoryId=4bf58dd8d48988d175941735'

Perform Get request 

In [7]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5f91b02d7b3a283282bfa220'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Downtown Manhattan',
  'headerFullLocation': 'Downtown Manhattan, New York',
  'headerLocationGranularity': 'neighborhood',
  'query': 'gym fitness',
  'totalResults': 88,
  'suggestedBounds': {'ne': {'lat': 40.7172281045, 'lng': -74.00008952063419},
   'sw': {'lat': 40.7082280955, 'lng': -74.0119408793658}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '58a253c830ecc66c9e5b40a0',
       'name': 'The Class by Taryn Toomey',
       'location': {'address': '22 Park Pl',
        'lat': 40.71275251771485,
        'lng': -74.00873355601571,
        'labeledLatLngs': [{'labe

All the data we need is located in the items key. To extract the category of a venue the function defined below will be used.

In [8]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [9]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

C:\Users\Wouter\anaconda3\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  This is separate from the ipykernel package so we can avoid doing imports until


,name,categories,lat,lng
0,The Class by Taryn Toomey,Gym / Fitness Center,40.712753,-74.008734
1,CrossFit 212 TriBeCa,Gym,40.714537,-74.005999
2,Exceed Physical Culture,Gym / Fitness Center,40.715629,-74.007992
3,Equinox Tribeca,Gym / Fitness Center,40.714099,-74.009686
4,The Helena Gym,Gym,40.714276,-74.005967


In [10]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

88 venues were returned by Foursquare.
